In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import concurrent.futures

In [8]:
# Constants, at earth

Ge = 6.674E-11
Me = 5.972E24
re = 6.371E6
ce = 3.00E8
Le = 0

#earth constant GM/rc2
K = 2*Ge*Me/(re*(ce)**2)
K

1.3902275240237882e-09

In [3]:
# small value to use instead of dividing by zero
epsilon = 1e-10

In [4]:
 #chart basis
basis = np.linspace(-0.999, 0.999, 100)

In [10]:
def gravity_dilation_eq(g,r,c,L): #relative to earth. Parameters are exponentials
    np.seterr(divide='ignore', invalid='ignore') 
    c= np.where(c==0, epsilon, c)
    return np.sqrt(1- K) / (epsilon + np.sqrt(1-K*g*r/c**2 + L*r/(3*c**2)) )
    


In [ ]:
def time_dilation(G,M,r,c,L) #parameters relative to earth
    np.seterr(divide='ignore', invalid='ignore') 
    c= np.where(c==0, epsilon, c)
    r= np.where(r==0, epsilon, r)
    denominator = np.where(1- K*G*M/(r*c**2) + L*r/(3*c**2))
    return np.sqrt( (1-K)/denominator )

In [ ]:
def g_force()

In [6]:
# Define function to be executed in parallel
def process_param(param,p_dict):
    new_p = p_dict.copy()
    new_p[param] = 10**(p_dict[param]+basis)
    return param, gravity_dilation_eq(**new_p)


# using exponential scales
def p_plot(g,r,c,L,zoom):
    # Clear the current figure
    plt.clf()

    # create a dictionary for the parameters
    p_dict = {'g': g, 'r': r, 'c':c, 'L': L}
    p_dict_exp = {'g': 10**g, 'r': 10**r, 'c':10**c, 'L':10**L}
    
    
    current_value= gravity_dilation_eq(**p_dict_exp)
    print(p_dict_exp,current_value)

    # empty dictionary for results
    
    gammas = {}
    
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Submit the tasks to the executor
        futures = {executor.submit(process_param, param, p_dict): param for param in p_dict}

        # Collect the results as they become available
        for future in concurrent.futures.as_completed(futures):
            param = futures[future]
            try:
                result_param, result_value = future.result()
                gammas[result_param] = result_value
                plt.plot(basis, result_value, label=result_param)
            except Exception as exc:
                print('%r generated an exception: %s' % (param, exc))
            
    # Plot current value
    plt.plot(0, current_value , 'ro')
    plt.annotate(round(current_value, 10), 
             (0,current_value),
             textcoords="offset points",  # how to position the text
             xytext=(-10,10),  # distance from text to points (x,y)
             ha='center',  # horizontal alignment can be left, right or center 
                )

    plt.xlabel('Parameter')
    plt.ylabel('Time dilation factor (gamma)')
    plt.legend()
#     plt.ylim(current_value*(1 - 10**-zoom),current_value*(1 + 10**-zoom))
 #   plt.yscale('log')
    plt.ylim(0,10**zoom)
    plt.grid(True)
    plt.show()

In [7]:
# Create interactive plot using exponential parameters
interactive_plot = interactive(p_plot, 
                               g=(-2, 2, 0.01), 
                               r=(-2, 2, 0.01), 
                               c=(-6, 6, 0.01), 
                               L=(-3, 3, 0.01), 
                               zoom=(1, 5, 1))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='g', max=2.0, min=-2.0, step=0.01), FloatSlider(value…